In [2]:
! pip install requests

  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)

   ---------------------------------------- 0/5 [urllib3]
   -------- ------------------------------- 1/5 [idna]
   -------------------------------- ------- 4/5 [requests]
   ---------------------------------------- 5/5 [requests]



## SCRAP DATA SD DI KABUPATEN SEMARANG LEWAT GMAPS

In [9]:
! pip install selenium

  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-2.0.0-cp310-cp310-win_amd64.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.23-py3-none-any.whl.metadata (993 bytes)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ------ --------------------------------- 1.6/9.6 MB 11.9 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.6 MB 13.2 MB/s eta 0:00:01
   --------------------------- ------------ 6.6/9.6 MB 12.2 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.6 MB 11.3 MB/s eta 0:00:01
   --------

In [ ]:
# from lib2to3.pgen2 import driver
# from pydoc import text
# import time
# import csv
# import json
# import re
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC


# # ==========================
# # 1️⃣ Setup Selenium
# # ==========================
# def setup_driver(headless=False):
#     options = Options()
#     if headless:
#         options.add_argument("--headless")
#     options.add_argument("--disable-gpu")
#     options.add_argument("--no-sandbox")
#     options.add_argument("--disable-dev-shm-usage")
#     options.add_argument("--window-size=1920,1080")
#     driver = webdriver.Chrome(options=options)
#     return driver


# # ==========================
# # 2️⃣ Load Daftar Kecamatan dari JSON
# # ==========================
# def load_kecamatan_list(json_path):
#     try:
#         with open(json_path, "r", encoding="utf-8") as f:
#             data = json.load(f)
#         kecamatan_list = data["Kabupaten Semarang"]["kecamatan"]
#         if isinstance(kecamatan_list[0], dict):
#             return [item["nama"] for item in kecamatan_list]
#         return kecamatan_list
#     except Exception as e:
#         print(f"⚠️ Gagal memuat data kecamatan: {e}")
#         return []


# # ==========================
# # 3️⃣ Scrape Maps Berdasarkan Query
# # ==========================
# def scrape_maps_by_query(kecamatan: str, kabupaten: str, json_path):
#     kecamatan = kecamatan.strip().upper()
#     kabupaten = kabupaten.strip().title()

#     daftar_kec = load_kecamatan_list(json_path)
#     if daftar_kec and kecamatan not in daftar_kec:
#         print(f"❌ Kecamatan '{kecamatan}' tidak ditemukan di Kabupaten {kabupaten}.")
#         print("📋 Kecamatan valid:", ", ".join(daftar_kec[:5]), "...")
#         return

#     kecamatan = kecamatan.strip().title()
#     query = f"SD di {kecamatan} Kabupaten {kabupaten}"
#     print(f"\n🔍 Mencari: {query}\n")

#     driver = setup_driver()
#     driver.get("https://www.google.com/maps")
#     time.sleep(3)

#     search_box = driver.find_element(By.ID, "searchboxinput")
#     search_box.clear()
#     search_box.send_keys(query)
#     search_box.send_keys(Keys.ENTER)
#     time.sleep(5)

#     print("🔄 Memuat semua hasil...")

#     # scroll list hasil
#     scrollable_div = driver.find_element(By.CSS_SELECTOR, "div[role='feed']")
#     last_height, same_count = 0, 0
#     while True:
#         driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
#         time.sleep(2)
#         new_height = driver.execute_script("return arguments[0].scrollTop", scrollable_div)
#         if new_height == last_height:
#             same_count += 1
#         else:
#             same_count = 0
#         if same_count >= 3:
#             break
#         last_height = new_height

#     # ambil semua href hasil
#     links = []
#     place_links = driver.find_elements(By.CSS_SELECTOR, "a.hfpxzc")
#     for pl in place_links:
#         href = pl.get_attribute("href")
#         if href and "maps/place" in href:
#             links.append(href)
            
#     print(f"✅ Ditemukan {len(links)} hasil.")

#     hasil = []

#     # loop ke setiap link (anti-refresh DOM)
#     for i, link in enumerate(links, start=1):
#         try:
#             driver.get(link)
#             WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "h1.DUwDvf"))
#             )
#             time.sleep(2)

#             # ambil nama sekolah
#             try:
#                 nama = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf").text
#             except:
#                 nama = "-"
                
#             # ambil alamat
#             try:
#                 alamat = driver.find_element(By.CSS_SELECTOR, "button[data-item-id*='address']").text
#                 text = alamat

#                 # 🧹 hilangkan semua karakter non-ASCII & simbol ikon Google (, dsb)
#                 text = re.sub(r"[^\x20-\x7E]+", "", text)

#                 # cek apakah mengandung pola Plus Code
#                 if re.match(r"^[A-Z0-9]{4,}\+[A-Z0-9]{2,}", text):
#                     # hapus plus code di depan, termasuk simbol, koma, dan spasi
#                     cleaned = re.sub(r"^[^\w]*[A-Z0-9]{4,}\+[A-Z0-9]{2,}[^\w,]*,?\s*", "", text)
#                     alamat = cleaned.strip()
#                 else:
#                     alamat = text.strip()

#             except:
#                 alamat = "-"

                
#             # ambil telepon/WA
#             try:
#                 telp = driver.find_element(By.CSS_SELECTOR, "button[data-item-id*='phone:tel']").text
#                 text = telp
                
#                  # 🧹 hilangkan semua karakter non-ASCII & simbol ikon Google (, dsb)
#                 text = re.sub(r"[^\x20-\x7E]+", "", text)
#                 telp = text.strip()
#             except:
#                 telp = "-"
                
#             # ambil website
#             try:
#                 situs = driver.find_element(By.CSS_SELECTOR, "a[data-item-id*='authority']").get_attribute("href")
#             except:
#                 situs = "-"
                
#             hasil.append({
#                 "Nama Sekolah": nama,
#                 "Alamat": alamat,
#                 "Telepon/WA": telp,
#                 "Website": situs,
#             })

#             # tombol kembali
#             back_btn = driver.find_elements(By.CSS_SELECTOR, "button[aria-label='Kembali ke hasil pencarian']")
#             if back_btn:
#                 back_btn[0].click()
#                 time.sleep(1.5)

#         except Exception:
#             continue

#     print("✅ Semua hasil telah dimuat.")

#     if not hasil:
#         print("⚠️ Tidak ditemukan hasil untuk query ini.")
#         return

#     filename = f"maps_sd_{kabupaten.lower()}_{kecamatan.lower()}.csv"
#     with open(filename, "w", newline="", encoding="utf-8") as f:
#         writer = csv.DictWriter(f, fieldnames=hasil[0].keys())
#         writer.writeheader()
#         writer.writerows(hasil)

#     print(f"💾 '{len(hasil)}' Data disimpan ke '{filename}'\n")
    
#     driver.quit()

# KEMENDIKBUD

In [12]:
import os
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC


def setup_driver(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--log-level=3")
    options.add_argument("--blink-settings=imagesEnabled=false")
    options.page_load_strategy = "eager"
    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver


def get_table_rows(driver, url):
    """Helper untuk ambil semua baris dari tabel referensi"""
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table#table1 tbody tr"))
    )
    try:
        select = Select(driver.find_element(By.NAME, "table1_length"))
        select.select_by_value("100")
    except:
        pass
    return driver.find_elements(By.CSS_SELECTOR, "table#table1 tbody tr")


def get_kecamatan_jateng_by_kode():
    """
    Ambil daftar kabupaten/kota di Jawa Tengah (030000),
    input kode kabupaten/kota (misal 032200),
    hasil disimpan ke data/kecamatan_<nama_kab>.json
    dalam format JSON hierarkis.
    """
    base_url = "https://referensi.data.kemendikdasmen.go.id/pendidikan/dikdas/"
    driver = setup_driver(headless=True)
    kecamatan_list = []

    try:
        # 1️⃣ Ambil daftar kabupaten/kota di Provinsi Jawa Tengah
        print("📦 Mengambil daftar kabupaten/kota di Provinsi Jawa Tengah...")
        kab_url = base_url + "030000/1"
        rows_kab = get_table_rows(driver, kab_url)

        kabupaten_list = []
        for r in rows_kab:
            try:
                nama = r.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
                href = r.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
                kode = href.split("/")[-2]
                kabupaten_list.append({"nama": nama, "kode": kode})
            except:
                continue

        print("\n📍 Daftar Kabupaten/Kota di Jawa Tengah:")
        print("=" * 60)
        for k in kabupaten_list:
            print(f"{k['kode']} - {k['nama']}")
        print("=" * 60)

        kode_input = input("\nMasukkan KODE kabupaten/kota yang ingin discrap (contoh: 032200): ").strip()

        kab = next((k for k in kabupaten_list if k["kode"] == kode_input), None)
        if not kab:
            print("❌ Kode tidak ditemukan dalam daftar.")
            driver.quit()
            return []

        print(f"\n📦 Mengambil daftar kecamatan di {kab['nama']}...")

        # 2️⃣ Ambil daftar kecamatan dari kabupaten/kota terpilih
        kec_url = base_url + f"{kab['kode']}/2"
        rows_kec = get_table_rows(driver, kec_url)

        for r in rows_kec:
            try:
                nama = r.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
                href = r.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
                kode = href.split("/")[-2]
                kecamatan_list.append({"nama": nama, "kode": kode, "url": href})
            except:
                continue

        print(f"✅ Berhasil ambil {len(kecamatan_list)} kecamatan dari {kab['nama']}")

        # 3️⃣ Simpan hasil ke data/
        os.makedirs("data", exist_ok=True)
        safe_name = kab["nama"].replace(" ", "_").replace(".", "").lower()
        save_path = os.path.join("data", f"kecamatan_{safe_name}.json")

        # === Format sesuai contoh Prof ===
        json_data = {
            kab["nama"]: {
                "kode": kab["kode"],
                "kecamatan": {k["nama"]: k["kode"] for k in kecamatan_list}
            }
        }

        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(json_data, f, ensure_ascii=False, indent=2)

        print(f"💾 Data disimpan ke: {save_path}")

    except Exception as e:
        print(f"❌ Terjadi kesalahan: {e}")

    driver.quit()
    return kecamatan_list


# ==========================
# 🚀 Jalankan
# ==========================
if __name__ == "__main__":
    kecamatan_data = get_kecamatan_jateng_by_kode()

📦 Mengambil daftar kabupaten/kota di Provinsi Jawa Tengah...

📍 Daftar Kabupaten/Kota di Jawa Tengah:
030100 - Kab. Cilacap
030200 - Kab. Banyumas
030300 - Kab. Purbalingga
030400 - Kab. Banjarnegara
030500 - Kab. Kebumen
030600 - Kab. Purworejo
030700 - Kab. Wonosobo
030800 - Kab. Magelang
030900 - Kab. Boyolali
031000 - Kab. Klaten
031100 - Kab. Sukoharjo
031200 - Kab. Wonogiri
031300 - Kab. Karanganyar
031400 - Kab. Sragen
031500 - Kab. Grobogan
031600 - Kab. Blora
031700 - Kab. Rembang
031800 - Kab. Pati
031900 - Kab. Kudus
032000 - Kab. Jepara
032100 - Kab. Demak
032200 - Kab. Semarang
032300 - Kab. Temanggung
032400 - Kab. Kendal
032500 - Kab. Batang
032600 - Kab. Pekalongan
032700 - Kab. Pemalang
032800 - Kab. Tegal
032900 - Kab. Brebes
036000 - Kota Magelang
036100 - Kota Surakarta
036200 - Kota Salatiga
036300 - Kota Semarang
036400 - Kota Pekalongan
036500 - Kota Tegal

📦 Mengambil daftar kecamatan di Kab. Semarang...
✅ Berhasil ambil 19 kecamatan dari Kab. Semarang
💾 Data di

In [ ]:
import json
import csv
import os
from pandas import options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC


# ==========================
# 🔧 Setup Driver
# ==========================
def setup_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--log-level=3")
    options.page_load_strategy = "eager"
    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

def get_table_rows(driver, url):
    """Helper untuk ambil semua baris dari tabel referensi"""
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table#table1 tbody tr"))
    )
    try:
        select = Select(driver.find_element(By.NAME, "table1_length"))
        select.select_by_value("100")
    except:
        pass
    return driver.find_elements(By.CSS_SELECTOR, "table#table1 tbody tr")

# ==========================
# 📂 Ambil Kode Kecamatan dari JSON
# ==========================
def get_kode_kecamatan_from_json(nama_kecamatan, json_path="./list_kecamatan/kecamatan_kab_semarang.json"):
    try:
        # Buka file JSON
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        if not data:
            print("❌ File JSON kosong.")
            return None

        nama_wilayah = next(iter(data))
        kabupaten_data = data[nama_wilayah].get("kecamatan", {})

        # Cari kecamatan
        for nama, kode in kabupaten_data.items():
            if nama.lower() == nama_kecamatan.lower():
                return kode

        print(f"❌ Kecamatan '{nama_kecamatan}' tidak ditemukan di {nama_wilayah}.")
        return None

    except FileNotFoundError:
        print(f"❌ File JSON '{json_path}' tidak ditemukan.")
        return None

    except Exception as e:
        print(f"⚠️ Error saat membaca JSON: {e}")
        return None



# ==========================
# 🔍 Ambil data Siswa Laki-laki, Kepsek, Alamat dari website sekolah kita kemendikbud
# ==========================
def get_detail(driver):
    alamat, kepsek, siswa_laki, siswa_perempuan = "-", "-", "-", "-"

    try:
        # Tunggu tab identitas muncul
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "label[for='tab-1']"))
        )
        rows_identitas = driver.find_elements(By.CSS_SELECTOR, "div.tabby-content table tr")

        for rp in rows_identitas:
            tds = rp.find_elements(By.CSS_SELECTOR, "td")
            if len(tds) < 4:
                continue

            link_elem = tds[3].find_elements(By.TAG_NAME, "a")
            if not link_elem:
                continue

            link_kemdikbud = link_elem[0].get_attribute("href")

            #  Buka halaman sekolah.data.kemdikbud.go.id
            driver.execute_script("window.open(arguments[0]);", link_kemdikbud)
            driver.switch_to.window(driver.window_handles[-1])

            try:
                # Tunggu halaman profil terbuka
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h4.page-header"))
                )

                #  Ambil alamat
                try:
                    alamat_elem = driver.find_element(By.CSS_SELECTOR, "font.small")
                    # Bersihkan teks tambahan seperti "(master referensi)"
                    alamat_elem = alamat_elem.replace("(master referensi)", "").text.strip()
                    alamat = alamat_elem if alamat_elem else "-"
                except:
                    alamat = "-"

                #  Ambil Kepala Sekolah
                try:
                    kepsek_elem = driver.find_element(
                        By.XPATH, "//li[contains(., 'Kepala Sekolah')]"
                    )
                    kepsek = kepsek_elem.text.split(":", 1)[-1].strip()
                except:
                    kepsek = "-"

                #  Ambil jumlah siswa laki-laki
                try:
                    siswa_laki_elem = driver.find_element(
                        By.XPATH,
                        "//text()[contains(., 'Siswa Laki-laki')]/following::font[1]"
                    )
                    siswa_laki = siswa_laki_elem.text.strip()
                except:
                    siswa_laki = "-"
                    
                # Ambil jumlah siswa perempuan 
                try:
                    siswa_perempuan_elem = driver.find_element(
                        By.XPATH,
                        "//text()[contains(., 'Siswa Perempuan')]/following::font[1]"
                    )
                    siswa_perempuan = siswa_perempuan_elem.text.strip()
                except:
                    siswa_perempuan = "-"

            except Exception as e:
                print(f"⚠️ Gagal ambil data di sekolah.data.kemdikbud.go.id: {e}")

            # Tutup tab & kembali ke tab utama
            driver.close()
            driver.switch_to.window(driver.window_handles[-1])
            break

    except Exception as e:
        print(f"⚠️ Gagal proses tab identitas: {e}")

    return alamat, kepsek, siswa_laki, siswa_perempuan


# ==========================
# ☎️ Ambil data kontak Referensi Data Kemendikbud
# ==========================
def get_kontak(driver):
    telepon, email, website = "-", "-", "-"
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "label[for='tab-4']"))
        )
        driver.find_element(By.CSS_SELECTOR, "label[for='tab-4']").click()
        rows_kontak = driver.find_elements(By.CSS_SELECTOR, "div.tabby-content table tr")

        for row in rows_kontak:
            tds = row.find_elements(By.CSS_SELECTOR, "td")
            if len(tds) < 4:
                continue
            label = tds[1].text.strip().lower()
            value = tds[3].text.strip() or "-"
            if "telepon" in label:
                telepon = value if len(value) >= 5 else "-"
            elif "email" in label:
                email = value
            elif "website" in label:
                val = value.lower()
                website = "-" if val in ["http://-", "https://-"] else val
    except:
        pass
    return telepon, email, website


# ==========================
# 🏫 Ambil Daftar Sekolah (SD & MI) dari Referensi Data Kemendikbud
# ==========================
def get_sd_mi_schools(kode_kecamatan, nama_kecamatan):
    driver = setup_driver(headless=False)
    sekolah_list = []

    for jenjang, value in [("SD", "5"), ("MI", "9")]:
        url = f"https://referensi.data.kemendikdasmen.go.id/pendidikan/dikdas/{kode_kecamatan}/3/all/{value}/all"
        print(f"🔗 Mengambil data {jenjang} dari: {url}")
        try:
            driver.get(url)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table#table1"))
            )

            # Ubah tampilan jadi 100 baris
            get_table_rows(driver, url)

            rows = driver.find_elements(By.CSS_SELECTOR, "table#table1 tbody tr")
            if not rows:
                print(f"⚠️ Tidak ada data {jenjang} di {nama_kecamatan}")
                continue

            for r in rows:
                try:
                    nama = r.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text.strip()
                    npsn = r.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
                    status = r.find_element(By.CSS_SELECTOR, "td:nth-child(6)").text.strip()
                    kelurahan = r.find_element(By.CSS_SELECTOR, "td:nth-child(5)").text.strip()
                    href = r.find_element(By.CSS_SELECTOR, "a").get_attribute("href")

                    # Buka halaman detail
                    driver.execute_script("window.open(arguments[0]);", href)
                    driver.switch_to.window(driver.window_handles[-1])

                    alamat, kepsek, siswa_laki, siswa_perempuan = get_detail(driver)
                    telepon, email, website = get_kontak(driver)

                    sekolah_list.append({
                        "Kelurahan": kelurahan,
                        "Nama Sekolah": nama,
                        "NPSN": npsn,
                        "Status": status,
                        "Kepala Sekolah": kepsek,
                        "Alamat": alamat,
                        "Telepon": telepon,
                        "Email": email,
                        "Website": website,
                        "Jumlah Siswa Laki-laki": siswa_laki,
                        "Jumlah Siswa Perempuan": siswa_perempuan,
                    })

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                except Exception as e:
                    print(f"⚠️ Gagal proses {nama}: {e}")
                    continue

            print(f"✅ {len(rows)} sekolah {jenjang} di {nama_kecamatan}")

        except Exception as e:
            print(f"❌ Gagal ambil data {jenjang} di {nama_kecamatan}: {e}")
            continue

    driver.quit()
    # 🧩 SORTING berdasarkan nama kelurahan
    sekolah_list = sorted(sekolah_list, key=lambda x: x["Kelurahan"].lower())
    return sekolah_list


# ==========================
# 💾 Simpan ke CSV
# ==========================
def save_school_list_by_kecamatan(nama_kecamatan):
    nama_kecamatan = nama_kecamatan.strip()
    kode_kecamatan = get_kode_kecamatan_from_json(nama_kecamatan)
    if not kode_kecamatan:
        return

    print(f"📍 Mengambil daftar SD & MI di Kecamatan {nama_kecamatan} (kode {kode_kecamatan})...")
    sekolah_list = get_sd_mi_schools(kode_kecamatan, nama_kecamatan)

    if not sekolah_list:
        print(f"❌ Tidak ditemukan SD atau MI di {nama_kecamatan}.")
        return

    folder = "output"
    os.makedirs(folder, exist_ok=True)
    filename = f"list_sd_mi_{nama_kecamatan.lower().replace(' ', '_')}.csv"
    fullpath = os.path.join(folder, filename)

    with open(fullpath, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=[
            "Kelurahan", "Nama Sekolah", "NPSN", "Status", "Kepala Sekolah", "Alamat", "Telepon", "Email", "Website", "Jumlah Siswa Laki-laki", "Jumlah Siswa Perempuan"
        ])
        writer.writeheader()
        writer.writerows(sekolah_list)

    print(f"✅ {len(sekolah_list)} sekolah disimpan ke '{fullpath}'")


# ==========================
# 🚀 Jalankan
# ==========================
if __name__ == "__main__":
    nama_kecamatan = "Ambarawa"
    save_school_list_by_kecamatan(nama_kecamatan)

📍 Mengambil daftar SD & MI di Kecamatan Ambarawa (kode 032210)...
🔗 Mengambil data SD dari: https://referensi.data.kemendikdasmen.go.id/pendidikan/dikdas/032210/3/all/5/all
✅ 29 sekolah SD di Ambarawa
🔗 Mengambil data MI dari: https://referensi.data.kemendikdasmen.go.id/pendidikan/dikdas/032210/3/all/9/all
✅ 5 sekolah MI di Ambarawa
✅ 34 sekolah disimpan ke 'output\list_sd_mi_ambarawa.csv'


# CHOSE DATA YANG MAU DISCRAP

In [9]:
! pip install gradio

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp310-cp310-win_amd64.whl.metadata (6.9 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/63.5 MB ? eta -:--:--
   - -------------------------------------- 1.8/63.5 MB 12.6 MB/s eta 0:00:05
   -- --------------------------

In [38]:
import os
import csv
import json
import time
import gradio as gr
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from time import sleep


# =====================================================
# 🔧 SETUP SELENIUM CHROME DRIVER
# =====================================================
def setup_driver(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--log-level=3")
    options.page_load_strategy = "eager"
    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver


def get_table_rows(driver, url):
    """Helper untuk ambil semua baris dari tabel referensi"""
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table#table1 tbody tr"))
    )
    try:
        select = Select(driver.find_element(By.NAME, "table1_length"))
        select.select_by_value("100")
    except:
        pass
    return driver.find_elements(By.CSS_SELECTOR, "table#table1 tbody tr")

# =====================================================
# 📂 AMBIL KODE KECAMATAN DARI JSON
# =====================================================
def get_kode_kecamatan_from_json(nama_kecamatan, json_path="./list_kecamatan/kecamatan_kab_semarang.json"):
    try:
        # Buka file JSON
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        if not data:
            print("❌ File JSON kosong.")
            return None

        nama_wilayah = next(iter(data))
        kabupaten_data = data[nama_wilayah].get("kecamatan", {})

        # Cari kecamatan
        for nama, kode in kabupaten_data.items():
            if nama.lower() == nama_kecamatan.lower():
                return kode

        print(f"❌ Kecamatan '{nama_kecamatan}' tidak ditemukan di {nama_wilayah}.")
        return None

    except FileNotFoundError:
        print(f"❌ File JSON '{json_path}' tidak ditemukan.")
        return None

    except Exception as e:
        print(f"⚠️ Error saat membaca JSON: {e}")
        return None

# =====================================================
# 🔍 DETAIL SEKOLAH (ALAMAT, KEPSEK, SISWA)
# =====================================================
def get_detail(driver):
    alamat, kepsek, siswa_laki, siswa_perempuan = "-", "-", "-", "-"
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "label[for='tab-1']"))
        )
        rows_identitas = driver.find_elements(By.CSS_SELECTOR, "div.tabby-content table tr")
        for rp in rows_identitas:
            tds = rp.find_elements(By.CSS_SELECTOR, "td")
            if len(tds) < 4:
                continue
            link_elem = tds[3].find_elements(By.TAG_NAME, "a")
            if not link_elem:
                continue
            link_kemdikbud = link_elem[0].get_attribute("href")

            # Buka tab baru
            driver.execute_script("window.open(arguments[0]);", link_kemdikbud)
            driver.switch_to.window(driver.window_handles[-1])

            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h4.page-header"))
                )

                # Alamat
                try:
                    alamat_elem = driver.find_element(By.CSS_SELECTOR, "font.small")
                    alamat_text = alamat_elem.text.replace("(master referensi)", "").strip()
                    alamat = alamat_text if alamat_text else "-"
                except:
                    alamat = "-"

                # Kepala Sekolah
                try:
                    kepsek_elem = driver.find_element(By.XPATH, "//li[contains(., 'Kepala Sekolah')]")
                    kepsek = kepsek_elem.text.split(":", 1)[-1].strip()
                except:
                    kepsek = "-"

                # Jumlah Siswa Laki-laki
                try:
                    siswa_laki_elem = driver.find_element( By.XPATH, "//text()[contains(., 'Siswa Laki-laki')]/following::font[1]")
                    siswa_laki = siswa_laki_elem.text.strip()
                except:
                    siswa_laki = "-"

                # Jumlah Siswa Perempuan
                try:
                    siswa_perempuan_elem = driver.find_element(By.XPATH, "//text()[contains(., 'Siswa Perempuan')]/following::font[1]")
                    siswa_perempuan = siswa_perempuan_elem.text.strip()
                except:
                    siswa_perempuan = "-"

            except Exception as e:
                print(f"⚠️ Gagal ambil data detail: {e}")

            driver.close()
            driver.switch_to.window(driver.window_handles[-1])
            break
    except Exception as e:
        print(f"⚠️ Gagal proses tab identitas: {e}")

    return alamat, kepsek, siswa_laki, siswa_perempuan


# =====================================================
# ☎️ KONTAK SEKOLAH
# =====================================================
def get_kontak(driver):
    telepon, email, website = "-", "-", "-"
    try:
        driver.find_element(By.CSS_SELECTOR, "label[for='tab-4']").click()
        sleep(0.5)
        rows_kontak = driver.find_elements(By.CSS_SELECTOR, "div.tabby-content table tr")
        for row in rows_kontak:
            tds = row.find_elements(By.CSS_SELECTOR, "td")
            if len(tds) < 4:
                continue
            label = tds[1].text.strip().lower()
            value = tds[3].text.strip() or "-"
            if "telepon" in label:
                telepon = value if len(value) >= 5 else "-"
            elif "email" in label:
                email = value
            elif "website" in label:
                val = value.lower()
                website = "-" if val in ["http://-", "https://-"] else val
    except:
        pass
    return telepon, email, website


# =====================================================
# 🏫 AMBIL DATA SEKOLAH (SD & MI)
# =====================================================
def get_sd_mi_schools(kode_kecamatan, nama_kecamatan, selected_fields, progress=gr.Progress()):
    driver = setup_driver(headless=True)
    sekolah_list = []

    need_detail = any(f in selected_fields for f in ["Alamat", "Kepala Sekolah", "Jumlah Siswa Laki-laki", "Jumlah Siswa Perempuan"])
    need_kontak = any(f in selected_fields for f in ["Telepon", "Email", "Website"])

    for jenjang, value in [("SD", "5"), ("MI", "9")]:
        url = f"https://referensi.data.kemendikdasmen.go.id/pendidikan/dikdas/{kode_kecamatan}/3/all/{value}/all"
        driver.get(url)
        sleep(0.5)
        print(f"🔗 Mengambil data {jenjang} di {nama_kecamatan}...")

        all_rows = []
        try:
            get_table_rows(driver, url)
            while True:
                rows = driver.find_elements(By.CSS_SELECTOR, "table#table1 tbody tr")
                all_rows.extend(rows)
                next_btn = driver.find_element(By.CSS_SELECTOR, "#table1_next")
                if "disabled" in next_btn.get_attribute("class"):
                    break
                driver.execute_script("arguments[0].click();", next_btn)
                sleep(1.2)
        except:
            continue

        total = len(all_rows)
        processed = 0

        for r in all_rows:
            data = {}
            try:
                if "Nama Sekolah" in selected_fields:
                    data["Nama Sekolah"] = r.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text.strip()
                if "NPSN" in selected_fields:
                    data["NPSN"] = r.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text.strip()
                if "Status" in selected_fields:
                    data["Status"] = r.find_element(By.CSS_SELECTOR, "td:nth-child(6)").text.strip()
                if "Kelurahan" in selected_fields:
                    data["Kelurahan"] = r.find_element(By.CSS_SELECTOR, "td:nth-child(5)").text.strip()

                if need_detail or need_kontak:
                    href = r.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
                    driver.execute_script("window.open(arguments[0]);", href)
                    driver.switch_to.window(driver.window_handles[-1])

                    if need_detail:
                        alamat, kepsek, siswa_laki, siswa_perempuan = get_detail(driver)
                        if "Alamat" in selected_fields: data["Alamat"] = alamat
                        if "Kepala Sekolah" in selected_fields: data["Kepala Sekolah"] = kepsek
                        if "Jumlah Siswa Laki-laki" in selected_fields: data["Jumlah Siswa Laki-laki"] = siswa_laki
                        if "Jumlah Siswa Perempuan" in selected_fields: data["Jumlah Siswa Perempuan"] = siswa_perempuan

                    if need_kontak:
                        telepon, email, website = get_kontak(driver)
                        if "Telepon" in selected_fields: data["Telepon"] = telepon
                        if "Email" in selected_fields: data["Email"] = email
                        if "Website" in selected_fields: data["Website"] = website

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                sekolah_list.append(data)
            except:
                pass
            processed += 1
            if total > 0:
                progress(processed / total)

    driver.quit()

    # ===== 🔢 SORTING berdasarkan prioritas =====
    sort_priority = [
        "Kelurahan", "Nama Sekolah", "NPSN", "Status", "Kepala Sekolah",
        "Alamat", "Telepon", "Email", "Website",
        "Jumlah Siswa Laki-laki", "Jumlah Siswa Perempuan"
    ]
    sort_key = next((f for f in sort_priority if f in selected_fields), None)
    if sort_key:
        sekolah_list.sort(key=lambda x: x.get(sort_key, "").lower())
    return sekolah_list


# =====================================================
# 💾 SIMPAN CSV
# =====================================================
def save_school_list_by_kecamatan(nama_kecamatan, selected_fields):
    kode_kecamatan = get_kode_kecamatan_from_json(nama_kecamatan)
    sekolah_list = get_sd_mi_schools(kode_kecamatan, nama_kecamatan, selected_fields)
    if not sekolah_list:
        return f"❌ Tidak ada data untuk '{nama_kecamatan}'.", None

    folder = "output"
    os.makedirs(folder, exist_ok=True)
    filename = f"list_sd_mi_{nama_kecamatan.lower().replace(' ', '_')}.csv"
    path = os.path.join(folder, filename)

    full_order = [
        "Kelurahan", "Nama Sekolah", "NPSN", "Status", "Kepala Sekolah",
        "Alamat", "Telepon", "Email", "Website",
        "Jumlah Siswa Laki-laki", "Jumlah Siswa Perempuan"
    ]
    ordered_fields = [col for col in full_order if col in selected_fields]

    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=ordered_fields)
        writer.writeheader()
        writer.writerows(sekolah_list)
    return f"✅ {len(sekolah_list)} sekolah disimpan ke '{path}'", path


# =====================================================
# 🚀 GRADIO UI
# =====================================================
def run_scraper(nama_kecamatan, selected_fields):
    if nama_kecamatan in ["", "-- Pilih Kecamatan --"]:
        yield "⚠️ Silakan pilih kecamatan terlebih dahulu.", gr.update(visible=False)
        return
    if not selected_fields:
        yield "⚠️ Pilih minimal satu kolom sebelum memulai scraping.", gr.update(visible=False)
        return

    start_time = time.time()
    status, file_path = save_school_list_by_kecamatan(nama_kecamatan, selected_fields)
    durasi = time.time() - start_time
    menit, detik = divmod(int(durasi), 60)
    waktu_str = f"\n⏱️ Waktu: {menit} menit {detik} detik" if menit else f"\n⏱️ Waktu: {detik} detik"

    if file_path:
        yield f"{status}{waktu_str}", gr.update(value=file_path, visible=True)
    else:
        yield f"{status}{waktu_str}", gr.update(visible=False)


def create_gradio_ui(json_path="./list_kecamatan/kecamatan_kab_semarang.json"):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    nama_wilayah = next(iter(data))
    kecamatan_list = sorted(data[nama_wilayah].get("kecamatan", {}).keys())

    fields = [
        "Kelurahan", "Nama Sekolah", "NPSN", "Status", "Kepala Sekolah",
        "Alamat", "Telepon", "Email", "Website",
        "Jumlah Siswa Laki-laki", "Jumlah Siswa Perempuan"
    ]

    with gr.Blocks(title="Scraper SD/MI Kabupaten Semarang") as demo:
        gr.Markdown("## 🏫 Scraper SD/MI Kabupaten Semarang")
        kecamatan = gr.Dropdown(
            label="Pilih Kecamatan",
            choices=["-- Pilih Kecamatan --"] + kecamatan_list,
            value="-- Pilih Kecamatan --",
            interactive=True
        )
        kolom = gr.CheckboxGroup(label="Pilih Kolom CSV", choices=fields)
        tombol = gr.Button("✅ Mulai Scrape")
        status = gr.Textbox(label="Status", lines=3)
        file = gr.File(label="📁 File CSV", interactive=False, visible=False)
        tombol.click(fn=run_scraper, inputs=[kecamatan, kolom], outputs=[status, file])
    return demo


ui = create_gradio_ui()
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


🔗 Mengambil data SD di Ungaran Barat...
